<a href="https://colab.research.google.com/github/sakshamtawakley/CC-Task/blob/main/cc_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas
import re
with open("timetable.log", "r") as f:
        log_data = f.readlines()


['2025/08/01 08:00:22 [47.15.69.30] POST /courses 200 473.604µs\n',
 '2025/08/01 08:00:22 [103.144.92.185] router: /courses\n',
 '2025/08/01 08:00:22 [103.144.92.185] router: /courses [2024A8PS0885P]\n',
 '2025/08/01 08:00:22 [103.144.92.185] router: /courses\n',
 '2025/08/01 08:00:22 [103.144.92.185] router: /courses [2024A8PS0885P]\n',
 '2025/08/01 08:00:22 [103.144.92.185] POST /courses 200 235.712µs\n',
 '2025/08/01 08:00:22 [103.144.92.185] POST /courses 200 400.851µs\n',
 '2025/08/01 08:00:22 [47.15.69.30] router: /courses\n',
 '2025/08/01 08:00:22 [47.15.69.30] router: /courses [2023B4A71019P]\n',
 '2025/08/01 08:00:22 [47.15.69.30] POST /courses 200 390.742µs\n',
 '2025/08/01 08:00:23 [103.144.92.199] router: /courses\n',
 '2025/08/01 08:00:23 [103.144.92.199] router: /courses [2023A4PS0694P]\n',
 '2025/08/01 08:00:23 [103.144.92.199] POST /courses 200 418.024µs\n',
 '2025/08/01 08:00:23 [103.144.92.199] router: /courses\n',
 '2025/08/01 08:00:23 [103.144.92.199] router: /cours

In [6]:
parsed_data=[]
total_api=0
end_point_section=0
end_point_courses=0
status_codes_200=0
status_codes_403=0
status_codes_404=0



In [5]:
pattern_user = re.compile(
    r"(?P<date>\d{4}/\d{2}/\d{2}) (?P<time>\d{2}:\d{2}:\d{2}) "
    r"\[(?P<ip>[^\]]+)\] router: (?P<path>\S+)(?: \[(?P<user_id>[^\]]+)\])?"
)

pattern_http = re.compile(
    r"(?P<date>\d{4}/\d{2}/\d{2}) (?P<time>\d{2}:\d{2}:\d{2}) "
    r"\[(?P<ip>[^\]]+)\] (?P<method>\S+) (?P<path>\S+) "
    r"(?P<status>\d+) (?P<duration>[\d\.]+(?:µs|ms))"
)

pattern_msg = re.compile(
    r"(?P<date>\d{4}/\d{2}/\d{2}) (?P<time>\d{2}:\d{2}:\d{2}) "
    r"\[(?P<ip>[^\]]+)\] --- (?P<message>.+) ---"
)

In [9]:
for i in log_data:
  i=i.strip()
  m1 = pattern_http.match(i)
  m2 = pattern_user.match(i)
  m3 = pattern_msg.match(i)

  if m1:
            total_api=total_api+1
            if m1["path"] == "/courses":
                end_point_courses=end_point_courses+1
            elif m1["path"] == "/section":
                end_point_courses=end_point_section+1
            if int(m1["status"])==200:
                status_codes_200=status_codes_200+1
            elif int(m1["status"])==403:
                status_codes_403=status_codes_403+1
            elif int(m1["status"])==404:
                status_codes_404=status_codes_404+1
            parsed_data.append({
                "date": m1["date"],
                "time": m1["time"],
                "ip": m1["ip"],
                "method": m1["method"],
                "path": m1["path"],
                "status": int(m1["status"]),
                "duration": m1["duration"],
                "user_id": None,
                "message": None
            })
  elif m2:
            total_api=total_api+1
            if m2["path"] == "/courses":
                end_point_courses=end_point_courses+1
            elif m2["path"] == "/section":
                end_point_courses=end_point_section+1
            parsed_data.append({
                "date": m2["date"],
                "time": m2["time"],
                "ip": m2["ip"],
                "method": None,
                "path": m2["path"],
                "status": None,
                "duration": None,
                "user_id": m2["user_id"],
                "message": None
            })
  elif m3:
            parsed_data.append({
                "date": m3["date"],
                "time": m3["time"],
                "ip": m3["ip"],
                "method": None,
                "path": None,
                "status": None,
                "duration": None,
                "user_id": None,
                "message": m3["message"]
            })
  else:
            # Unmatched lines
            parsed_data.append({
                "date": None, "time": None, "ip": None,
                "method": None, "path": None, "status": None,
                "duration": None, "user_id": None, "message": i
            })


In [11]:
parsed_data


[{'date': '2025/08/01',
  'time': '08:00:22',
  'ip': '47.15.69.30',
  'method': 'POST',
  'path': '/courses',
  'status': 200,
  'duration': '473.604µs',
  'user_id': None,
  'message': None},
 {'date': '2025/08/01',
  'time': '08:00:22',
  'ip': '103.144.92.185',
  'method': None,
  'path': '/courses',
  'status': None,
  'duration': None,
  'user_id': None,
  'message': None},
 {'date': '2025/08/01',
  'time': '08:00:22',
  'ip': '103.144.92.185',
  'method': None,
  'path': '/courses',
  'status': None,
  'duration': None,
  'user_id': '2024A8PS0885P',
  'message': None},
 {'date': '2025/08/01',
  'time': '08:00:22',
  'ip': '103.144.92.185',
  'method': None,
  'path': '/courses',
  'status': None,
  'duration': None,
  'user_id': None,
  'message': None},
 {'date': '2025/08/01',
  'time': '08:00:22',
  'ip': '103.144.92.185',
  'method': None,
  'path': '/courses',
  'status': None,
  'duration': None,
  'user_id': '2024A8PS0885P',
  'message': None},
 {'date': '2025/08/01',
  't